# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
SMALL_KGE_DIMENSION = 4
LARGE_KGE_DIMENSION = MODEL_HIDDEN_SIZE
KGE_DIMENSIONS = [SMALL_KGE_DIMENSION, LARGE_KGE_DIMENSION] # Output Dimension of the GNN Encoder.
EPOCHS = 20
BATCH_SIZE = 256

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val,"[-1.1001030206680298, 1.8915419578552246, 1.12...","[-0.5872145891189575, 0.7743701934814453, 1.61...","[0.08414871990680695, -0.8586543798446655, -0....","[0.16821661591529846, -0.092796690762043, -0.3..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,train,"[-1.1077896356582642, 2.1780264377593994, 1.23...","[0.09496331214904785, 0.2714781165122986, 1.88...","[0.05774740129709244, -0.9950929284095764, -0....","[0.490773469209671, 0.3398897647857666, -0.477..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,train,"[-1.2368007898330688, 1.965051293373108, 1.000...","[-0.31836411356925964, -0.3369845151901245, 2....","[0.12886668741703033, -0.9324935674667358, -0....","[0.04016011953353882, 0.29606175422668457, -0...."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,train,"[-1.1827130317687988, 2.0680408477783203, 1.21...","[-1.0212271213531494, 0.4136617183685303, 2.13...","[0.007488217204809189, -0.8364493250846863, -0...","[0.08211947977542877, 0.01935999095439911, -0...."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",test,"[-1.0296814441680908, 2.030970811843872, 1.094...","[-0.7692033052444458, 0.5881525874137878, 1.53...","[0.07757449150085449, -0.9171958565711975, -0....","[0.1306394338607788, 0.0786743313074112, -0.00..."


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainer =    GNNTrainer(movie_lens_loader.data, kge_dimension = SMALL_KGE_DIMENSION)
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cuda'
loading pretrained model
Device: 'cuda'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:

print(gnn_trainer.kge_dimension)
#gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
print(gnn_trainer_large.kge_dimension)
#gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4


100%|██████████| 79/79 [00:04<00:00, 17.09it/s]



Validation AUC: 0.9342
128


100%|██████████| 79/79 [00:04<00:00, 17.10it/s]


Validation AUC: 0.9280


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
gnn_trainer.get_embeddings(movie_lens_loader)
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val,"[-1.1001030206680298, 1.8915419578552246, 1.12...","[-0.5872145891189575, 0.7743701934814453, 1.61...","[0.08414871990680695, -0.8586543798446655, -0....","[0.16821661591529846, -0.092796690762043, -0.3..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,train,"[-1.1077896356582642, 2.1780264377593994, 1.23...","[0.09496331214904785, 0.2714781165122986, 1.88...","[0.05774740129709244, -0.9950929284095764, -0....","[0.490773469209671, 0.3398897647857666, -0.477..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,train,"[-1.2368007898330688, 1.965051293373108, 1.000...","[-0.31836411356925964, -0.3369845151901245, 2....","[0.12886668741703033, -0.9324935674667358, -0....","[0.04016011953353882, 0.29606175422668457, -0...."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,train,"[-1.1827130317687988, 2.0680408477783203, 1.21...","[-1.0212271213531494, 0.4136617183685303, 2.13...","[0.007488217204809189, -0.8364493250846863, -0...","[0.08211947977542877, 0.01935999095439911, -0...."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",test,"[-1.0296814441680908, 2.030970811843872, 1.094...","[-0.7692033052444458, 0.5881525874137878, 1.53...","[0.07757449150085449, -0.9171958565711975, -0....","[0.1306394338607788, 0.0786743313074112, -0.00..."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension)

We also train the model. This can be skipped if already done ones.

In [13]:
prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS)

  0%|          | 0/4420 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.679, 'grad_norm': 2.0133321285247803, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.6778, 'grad_norm': 2.2803542613983154, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.673, 'grad_norm': 1.5807524919509888, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.6673, 'grad_norm': 1.5122690200805664, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.6604, 'grad_norm': 1.2263641357421875, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.654, 'grad_norm': 1.9036740064620972, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.6471, 'grad_norm': 1.2255926132202148, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.6484, 'grad_norm': 0.3636559844017029, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.6453, 'grad_norm': 0.5044724941253662, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.6391, 'grad_norm': 0.8040212988853455, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.6327, 'grad_norm': 0

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.6177912354469299, 'eval_accuracy': 0.6661999766654999, 'eval_runtime': 166.6224, 'eval_samples_per_second': 102.879, 'eval_steps_per_second': 0.402, 'epoch': 1.0}
{'loss': 0.625, 'grad_norm': 0.40551435947418213, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.04}
{'loss': 0.6142, 'grad_norm': 0.41211628913879395, 'learning_rate': 2.4e-05, 'epoch': 1.09}
{'loss': 0.6158, 'grad_norm': 0.9771807193756104, 'learning_rate': 2.5e-05, 'epoch': 1.13}
{'loss': 0.6055, 'grad_norm': 0.33162227272987366, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.18}
{'loss': 0.6205, 'grad_norm': 0.8342613577842712, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.22}
{'loss': 0.622, 'grad_norm': 0.8178257942199707, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.27}
{'loss': 0.6343, 'grad_norm': 0.9120437502861023, 'learning_rate': 2.9e-05, 'epoch': 1.31}
{'loss': 0.6142, 'grad_norm': 0.8860794901847839, 'learning_rate': 3e-05, 'epoch': 1.36}
{'loss': 0.6226, 'grad_norm': 0.56

...

{'eval_loss': 0.29272645711898804, 'eval_accuracy': 0.8714852409287132, 'eval_runtime': 170.2087, 'eval_samples_per_second': 100.712, 'eval_steps_per_second': 0.394, 'epoch': 18.0}
{'loss': 0.326, 'grad_norm': 1.4779797792434692, 'learning_rate': 5.612244897959184e-06, 'epoch': 18.01}
{'loss': 0.3435, 'grad_norm': 1.8217583894729614, 'learning_rate': 5.48469387755102e-06, 'epoch': 18.05}
{'loss': 0.338, 'grad_norm': 1.8670899868011475, 'learning_rate': 5.357142857142857e-06, 'epoch': 18.1}
{'loss': 0.3447, 'grad_norm': 2.4358131885528564, 'learning_rate': 5.229591836734695e-06, 'epoch': 18.14}
{'loss': 0.3281, 'grad_norm': 2.4266974925994873, 'learning_rate': 5.102040816326531e-06, 'epoch': 18.19}
{'loss': 0.3206, 'grad_norm': 3.318112373352051, 'learning_rate': 4.9744897959183674e-06, 'epoch': 18.24}
{'loss': 0.3292, 'grad_norm': 6.297029495239258, 'learning_rate': 4.846938775510204e-06, 'epoch': 18.28}
{'loss': 0.3276, 'grad_norm': 2.546241044998169, 'learning_rate': 4.71938775510204

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.2934187054634094, 'eval_accuracy': 0.8725352934313383, 'eval_runtime': 168.023, 'eval_samples_per_second': 102.022, 'eval_steps_per_second': 0.399, 'epoch': 19.0}
{'loss': 0.3211, 'grad_norm': 1.892685055732727, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.3317, 'grad_norm': 2.6029622554779053, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.3508, 'grad_norm': 2.1942896842956543, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.351, 'grad_norm': 2.260469913482666, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.3403, 'grad_norm': 2.1681981086730957, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.3134, 'grad_norm': 2.7360994815826416, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.3426, 'grad_norm': 3.0878658294677734, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.3226, 'grad_norm': 1.561689853668213, 'learning_rate': 1.9132653061224

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.28821152448654175, 'eval_accuracy': 0.8773188659432971, 'eval_runtime': 169.1877, 'eval_samples_per_second': 101.319, 'eval_steps_per_second': 0.396, 'epoch': 20.0}
{'train_runtime': 14044.0232, 'train_samples_per_second': 80.417, 'train_steps_per_second': 0.315, 'train_loss': 0.39407227238918324, 'epoch': 20.0}


In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)


Some weights of InsertEmbeddingBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/56469 [00:00<?, ? examples/s]

Map:   0%|          | 0/17142 [00:00<?, ? examples/s]

Map:   0%|          | 0/17142 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17142 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17142 [00:00<?, ? examples/s]

In [15]:
adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

  0%|          | 0/4420 [00:00<?, ?it/s]

{'loss': 0.7091, 'grad_norm': 2.509500026702881, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.7048, 'grad_norm': 2.799487590789795, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.7035, 'grad_norm': 2.143673896789551, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.6954, 'grad_norm': 2.0925886631011963, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.6853, 'grad_norm': 1.866171956062317, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.6781, 'grad_norm': 2.5431997776031494, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.6691, 'grad_norm': 1.8883814811706543, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.6606, 'grad_norm': 0.9473239779472351, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.6513, 'grad_norm': 0.37495705485343933, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.6463, 'grad_norm': 1.2949178218841553, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.6336, 'grad_norm': 0

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.4853430688381195, 'eval_accuracy': 0.7964648232411621, 'eval_runtime': 148.5075, 'eval_samples_per_second': 115.429, 'eval_steps_per_second': 0.451, 'epoch': 1.0}
{'loss': 0.517, 'grad_norm': 0.9431381821632385, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.04}
{'loss': 0.4844, 'grad_norm': 1.1250807046890259, 'learning_rate': 2.4e-05, 'epoch': 1.09}
{'loss': 0.4854, 'grad_norm': 1.1480079889297485, 'learning_rate': 2.5e-05, 'epoch': 1.13}
{'loss': 0.4693, 'grad_norm': 1.0225285291671753, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.18}
{'loss': 0.4675, 'grad_norm': 1.0478017330169678, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.22}
{'loss': 0.458, 'grad_norm': 1.0481812953948975, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.27}
{'loss': 0.4643, 'grad_norm': 1.0488988161087036, 'learning_rate': 2.9e-05, 'epoch': 1.31}
{'loss': 0.4418, 'grad_norm': 0.9810708165168762, 'learning_rate': 3e-05, 'epoch': 1.36}
{'loss': 0.4526, 'grad_norm': 1.03173

  0%|          | 0/67 [00:00<?, ?it/s]

...

{'eval_loss': 0.2536170184612274, 'eval_accuracy': 0.8919029284797573, 'eval_runtime': 152.5274, 'eval_samples_per_second': 112.386, 'eval_steps_per_second': 0.439, 'epoch': 18.0}
{'loss': 0.2626, 'grad_norm': 1.0358753204345703, 'learning_rate': 5.612244897959184e-06, 'epoch': 18.01}
{'loss': 0.2939, 'grad_norm': 0.8462299108505249, 'learning_rate': 5.48469387755102e-06, 'epoch': 18.05}
{'loss': 0.2691, 'grad_norm': 0.8137834668159485, 'learning_rate': 5.357142857142857e-06, 'epoch': 18.1}
{'loss': 0.269, 'grad_norm': 0.9100592136383057, 'learning_rate': 5.229591836734695e-06, 'epoch': 18.14}
{'loss': 0.2873, 'grad_norm': 1.3644741773605347, 'learning_rate': 5.102040816326531e-06, 'epoch': 18.19}
{'loss': 0.2681, 'grad_norm': 1.3821933269500732, 'learning_rate': 4.9744897959183674e-06, 'epoch': 18.24}
{'loss': 0.268, 'grad_norm': 1.9112229347229004, 'learning_rate': 4.846938775510204e-06, 'epoch': 18.28}
{'loss': 0.2714, 'grad_norm': 0.9123752117156982, 'learning_rate': 4.719387755102

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.24508707225322723, 'eval_accuracy': 0.8957531209893828, 'eval_runtime': 152.5009, 'eval_samples_per_second': 112.406, 'eval_steps_per_second': 0.439, 'epoch': 19.0}
{'loss': 0.2616, 'grad_norm': 1.0053839683532715, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.2901, 'grad_norm': 0.9918547868728638, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.2841, 'grad_norm': 0.9265093207359314, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.281, 'grad_norm': 0.8319127559661865, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.2923, 'grad_norm': 1.1985414028167725, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.2723, 'grad_norm': 0.8562483787536621, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.2864, 'grad_norm': 1.2020838260650635, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.269, 'grad_norm': 0.8529525995254517, 'learning_rate': 1.913265306

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.24707971513271332, 'eval_accuracy': 0.8940030334850075, 'eval_runtime': 152.7639, 'eval_samples_per_second': 112.212, 'eval_steps_per_second': 0.439, 'epoch': 20.0}
{'train_runtime': 13225.0559, 'train_samples_per_second': 85.397, 'train_steps_per_second': 0.334, 'train_loss': 0.3273738372379838, 'epoch': 20.0}
